In [1]:
from keras.applications.nasnet import NASNetLarge
from keras.applications.nasnet import preprocess_input
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.optimizers import Adam
import os
import numpy as np
import pandas as pd
import glob
from PIL import Image
path = '..'

Using TensorFlow backend.


### Model prep

In [2]:
labels = pd.read_csv(os.path.join('ground_truth_xml.csv'), sep=';')

In [3]:
_map = dict([el for el in enumerate(labels.loc[~labels['name'].str.contains('label_'), 'name'].value_counts().reset_index()['index'].values)])

In [4]:
_map = dict(zip(_map.values(), _map.keys()))

In [5]:
model_x = np.ndarray([1097, 331, 331, 3])
model_y = np.ndarray([1097, 67])

for pic in enumerate(glob.glob(os.path.join(r'input', r'ground_truth_items_r', '*.jpg'))):
    im1 = Image.open(pic[1])
    model_x[pic[0]] = np.array(im1)
    y_index = _map[pic[1].split('/')[-1].split('_')[0]]
    model_y[pic[0], y_index] = 1

In [6]:
_map

{'03001220': 0,
 '00050886': 1,
 '00050887': 2,
 '00050965': 3,
 '00062541': 4,
 '00062447': 5,
 '00062538': 6,
 '02802507': 7,
 '00062412': 8,
 '00062394': 9,
 '00060885': 10,
 '00062434': 11,
 '00062537': 12,
 '00062396': 13,
 '00057855': 14,
 '00062397': 15,
 '00049295': 16,
 '00062435': 17,
 '00062390': 18,
 '00066417': 19,
 '00015906': 20,
 '00057856': 21,
 '00073709': 22,
 '00052560': 23,
 '00070845': 24,
 '00052559': 25,
 '03001211': 26,
 '02802510': 27,
 '00062436': 28,
 '00062395': 29,
 '00073597': 30,
 '00073675': 31,
 '00032190': 32,
 '00049378': 33,
 '00062413': 34,
 '00070844': 35,
 '02802511': 36,
 '03100931': 37,
 '00052556': 38,
 '00054501': 39,
 '00066416': 40,
 '00062393': 41,
 '00015907': 42,
 '00052557': 43,
 '00020162': 44,
 '00014561': 45,
 '00021624': 46,
 '00069581': 47,
 '03001210': 48,
 '00021105': 49,
 '00052558': 50,
 '00030453': 51,
 '00060691': 52,
 '00033379': 53,
 '00069582': 54,
 '00060668': 55,
 '00030424': 56,
 '00011325': 57,
 '00015905': 58,
 '00049

### Model

In [7]:
img_rows, img_cols, img_channels = 331, 331, 3

In [8]:
base_model = NASNetLarge((img_rows, img_cols, img_channels), include_top=False, weights='imagenet')

Instructions for updating:
Colocations handled automatically by placer.
343613440/343610240 [==============================] - 273s 1us/step


In [9]:
#base_model.trainable = False
for layer in base_model.layers:
    layer.trainable = False

In [10]:
x = base_model.output
x = Dense(10, activation='softmax')(x)
x = Flatten()(x)
x = Dense(20, activation='softmax')(x)
preds = Dense(67, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=preds)

In [11]:
model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 331, 331, 3)  0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 165, 165, 96) 2592        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 165, 165, 96) 384         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 165, 165, 96) 0           stem_bn1[0][0]                   
__________________________________________________________________________________________________
reduction_

separable_conv_2_bn_normal_left (None, 42, 42, 168)  672         separable_conv_2_normal_left1_4[0
__________________________________________________________________________________________________
separable_conv_2_bn_normal_righ (None, 42, 42, 168)  672         separable_conv_2_normal_right1_4[
__________________________________________________________________________________________________
separable_conv_2_bn_normal_left (None, 42, 42, 168)  672         separable_conv_2_normal_left2_4[0
__________________________________________________________________________________________________
separable_conv_2_bn_normal_righ (None, 42, 42, 168)  672         separable_conv_2_normal_right2_4[
__________________________________________________________________________________________________
normal_left3_4 (AveragePooling2 (None, 42, 42, 168)  0           normal_bn_1_4[0][0]              
__________________________________________________________________________________________________
normal_lef

__________________________________________________________________________________________________
activation_166 (Activation)     (None, 21, 21, 2016) 0           normal_concat_11[0][0]           
__________________________________________________________________________________________________
adjust_conv_projection_12 (Conv (None, 21, 21, 336)  677376      activation_165[0][0]             
__________________________________________________________________________________________________
normal_conv_1_12 (Conv2D)       (None, 21, 21, 336)  677376      activation_166[0][0]             
__________________________________________________________________________________________________
adjust_bn_12 (BatchNormalizatio (None, 21, 21, 336)  1344        adjust_conv_projection_12[0][0]  
__________________________________________________________________________________________________
normal_bn_1_12 (BatchNormalizat (None, 21, 21, 336)  1344        normal_conv_1_12[0][0]           
__________

In [ ]:
model.fit(model_x, model_y, batch_size=64, epochs=50)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/50


In [ ]:
a = model.predict(model_x, verbose=1)